## Disparity Map

In [ ]:
r"""
 @file DisparityMap_hli038.ipynb
 @brief A tity program that implement the basic function of 
         computing disparity maps of two given images.
 
 [AI6121] Computer Vision - Assignment 2 - Disparity Map
 
 @author Hantao Li  <hli038@e.ntu.edu.sg>
 @date 01/10/2021
 """

import time
import numpy
import math
import os.path
import numpy as np
import cv2
from matplotlib import pyplot as plt

#Used when checking the SAD/SSD/NCC Value
BIG_VALUE = float('inf')
ZERO_VALUE = 0

#### Functions for Dispartiy computing

In [ ]:
# sobel filtering for preprocessing
def SobelFilter(image):
    height, width = image.shape
    out_image = numpy.zeros((height, width))

    table_x = numpy.array(([-1, -2, -1], [0, 0, 0], [1, 2, 1]))
    table_y = numpy.array(([-1, 0, 1], [-2, 0, 2], [-1, 0, 1]))

    for y in range(2, width - 2):
        for x in range(2, height - 2):
            cx, cy = 0, 0
            for offset_y in range(0, 3):
                for offset_x in range(0, 3):
                    pix = image[x + offset_x - 1, y + offset_y - 1]
                    if offset_x != 1:
                        cx += pix * table_x[offset_x, offset_y]
                    if offset_y != 1:
                        cy += pix * table_y[offset_x, offset_y]
            out_pix = math.sqrt(cx**2 + cy**2)
            out_image[x, y] = out_pix if out_pix > 0 else 0
    numpy.putmask(out_image, out_image > 255, 255)
    return out_image


def PreProcessing(gray_left, gray_right):
    sobel_left = SobelFilter(gray_left)
    sobel_right = SobelFilter(gray_right)
    return sobel_left, sobel_right


def CalcSAD(target_block, crspd_block):
    return (sum(sum(abs(target_block - crspd_block))))


def CalcSSD(target_block, crspd_block):
    return (sum(sum(abs(target_block - crspd_block)**2)))


def CalcNCC(target_block, crspd_block):
    diff_tar = target_block - np.mean(target_block)
    diff_crs = crspd_block - np.mean(crspd_block)
    NCC_up = sum(sum(diff_tar * diff_crs))
    NCC_down = math.sqrt(sum(sum(diff_tar**2)) * sum(sum(diff_crs**2)))
    return (NCC_up / NCC_down)


def MatchImage(target_block, crspd_block, matching_method):
    if (matching_method == 'SAD'):
        result = CalcSAD(target_block, crspd_block)
    elif (matching_method == 'SSD'):
        result = CalcSSD(target_block, crspd_block)
    elif (matching_method == 'NCC'):
        result = CalcNCC(target_block, crspd_block)
    else:
        print('Error! Please check the matching method!')
        exit(0)
    return result


def AssignDirct(gray_left, gray_right, direction):
    if (direction == 'Left'):
        target_image = gray_left
        crspd_image = gray_right
        print('Start Drawing Left Disparity Map...')
    elif (direction == 'Right'):
        target_image = gray_right
        crspd_image = gray_left
        print('Start Drawing Right Disparity Map...')
    else:
        print('Error! Please check the Direction!')
        exit(0)
    return target_image, crspd_image


def CalcDEdge(j, half_block, width, direction):
    if (direction == 'Left'):
        edge_value = j - half_block - 1
    elif (direction == 'Right'):
        edge_value = width - j - half_block
    return edge_value


def CalcBlockEdge(i, j, half_block, d, direction):
    up_edge = i - half_block
    below_dege = i + half_block
    if (direction == 'Left'):
        left_edge = j - half_block - d
        right_edge = j + half_block - d
    elif (direction == 'Right'):
        left_edge = j - half_block + d
        right_edge = j + half_block + d

    return up_edge, below_dege, left_edge, right_edge


def AssignBar(matching_method):
    if ((matching_method == 'SSD') or (matching_method == 'SAD')):
        init_bar = BIG_VALUE
    elif (matching_method == 'NCC'):
        init_bar = ZERO_VALUE
    init_dist = ZERO_VALUE

    return init_bar, init_dist


def UpdateVal(bar, dist, Val, d, matching_method):
    if ((matching_method == 'SSD') or (matching_method == 'SAD')):
        if Val < bar:
            bar = Val
            dist = d
    elif (matching_method == 'NCC'):
        if Val > bar:
            bar = Val
            dist = d

    return bar, dist


# Calculate disparity
def CalcDisparity(gray_left, gray_right, num_disparity, block_size,
                  matching_method, direction):

    #Check the input direction
    target_image, crspd_iamge = AssignDirct(gray_left, gray_right, direction)

    #Check the input images' shape
    if (target_image.shape == crspd_iamge.shape):
        height, width = target_image.shape
    else:
        print('Error! Two Image have different shape!')
        exit(0)

    disparity_matrix = numpy.zeros((height, width), dtype=numpy.float32)
    half_block = block_size // 2

    #Load the range for kernel
    for i in range(half_block, height - half_block):
        print("%d%% " % (i * 100 // height), end=' ', flush=True)

        for j in range(half_block, width - half_block):
            target_block = target_image[i - half_block:i + half_block,
                                        j - half_block:j + half_block]

            bar, dist = AssignBar(matching_method)
            edge_value = CalcDEdge(j, half_block, width, direction)

            for d in range(0, min(edge_value, num_disparity)):
                up_edge, below_dege, left_edge, right_edge = CalcBlockEdge(
                    i, j, half_block, d, direction)
                crspd_block = crspd_iamge[up_edge:below_dege,
                                          left_edge:right_edge]

                Val = MatchImage(target_block, crspd_block, matching_method)
                bar, dist = UpdateVal(bar, dist, Val, d, matching_method)

            disparity_matrix[i - half_block, j - half_block] = dist
    print('100%')
    return disparity_matrix


# Left-right verification
def Postprocessing(disp_left, disp_right):
    height, width = disp_left.shape
    out_image = disp_left

    for h in range(1, height - 1):
        for w in range(1, width - 1):
            left = int(disp_left[h, w])
            if w - left > 0:
                right = int(disp_right[h, w - left])
                dispDiff = left - right
                if dispDiff < 0:
                    dispDiff = -dispDiff
                elif dispDiff > 1:
                    out_image[h, w] = 0
    return out_image

In [ ]:
def DisprityMap(left_image_path, right_image_path, matching_method,
                num_disparity, block_size, work_name, save_path):

    second_path = work_name + matching_method + ' ' + str(
        num_disparity) + ' ' + str(block_size)

    print('-----------')
    print('Start......')
    print(second_path)
    print('-----------')
    start_time = time.time()

    # Read left and right images
    image_left = cv2.imread(os.path.join(left_image_path))
    image_right = cv2.imread(os.path.join(right_image_path))

    # Convert to grayscale
    gray_left = numpy.mean(image_left, 2)
    gray_right = numpy.mean(image_right, 2)

    # Preprocessing
    sobel_left, sobel_right = PreProcessing(gray_left, gray_right)

    disparity_left = CalcDisparity(sobel_left, sobel_right, num_disparity,
                                   block_size, matching_method, 'Left')
    disparity_left_color = cv2.applyColorMap(
        cv2.convertScaleAbs(disparity_left, alpha=256 / num_disparity),
        cv2.COLORMAP_JET)
    cv2.imwrite(save_path + second_path + ' Left.bmp', disparity_left_color)

    disparity_right = CalcDisparity(sobel_left, sobel_right, num_disparity,
                                    block_size, matching_method, 'Right')
    disparity_right_color = cv2.applyColorMap(
        cv2.convertScaleAbs(disparity_right, alpha=256 / num_disparity),
        cv2.COLORMAP_JET)
    cv2.imwrite(save_path + second_path + ' Right.bmp', disparity_right_color)

    print('-----------')
    print('Postprocessing...')
    print('-----------')
    disparity = Postprocessing(disparity_left, disparity_right)

    # Generate color image and save to file
    disparity_color = cv2.applyColorMap(
        cv2.convertScaleAbs(disparity, alpha=256 / num_disparity),
        cv2.COLORMAP_JET)
    cv2.imwrite(save_path + second_path + '.bmp', disparity_color)

    print('Duration: %s seconds\n' % (time.time() - start_time))

#### Computing procedure

In [ ]:
#left_image_path = './Input/corridorl.jpg'
#right_image_path = './Input/corridorr.jpg'

left_image_path = './Input/triclopsi2l.jpg'
right_image_path = './Input/triclopsi2r.jpg'

#block_size_list = [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35]
#block_size_list = [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]
block_size_list = [27, 29, 31, 33, 35]# Size of Kernel
matching_method_list = ['SSD','SAD','NCC']

num_disparity = 32  # 视差范围
work_name = 'Triccccc-New'
save_path = './Figure/'

for matching_method in matching_method_list:
    for block_size in block_size_list:
        DisprityMap(left_image_path, right_image_path, matching_method,
                    num_disparity, block_size, work_name, save_path)

#### SGBM, call OpenCV

In [ ]:
imgL = cv2.imread(r'./Input/triclopsi2l.jpg')
imgR = cv2.imread(r'./Input/triclopsi2r.jpg')
#imgL = cv2.imread(r'./Input/corridorl.jpg')
#imgR = cv2.imread(r'./Input/corridorr.jpg')

# disparity range tuning
window_size = 13
min_disp = 16
num_disp = 320 - min_disp

stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=16,  # max_disp has to be dividable by 16 f. E. HH 192, 256
    blockSize=5,
    P1=8 * 3 * window_size ** 2,
    # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
    P2=32 * 3 * window_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=15,
    speckleWindowSize=0,
    speckleRange=2,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)
disparity = stereo.compute(imgL, imgR).astype(np.float32) / 16.0
fig = plt.figure(constrained_layout=False, dpi=600)
plt.imshow(disparity, 'gray')
fig.savefig("./232asdf23.png",dpi=600)
#plt.jet()
plt.show()

#### BM, call OpenCV

In [ ]:
imgL_1 = cv2.imread(r'./Input/triclopsi2l.jpg')
imgR_1 = cv2.imread(r'./Input/triclopsi2r.jpg')

#imgL_1 = cv2.imread(r'./Input/corridorl.jpg')
#imgR_1 = cv2.imread(r'./Input/corridorR.jpg')

imgL = cv2.cvtColor(imgL_1, cv2.COLOR_BGR2GRAY)
imgR = cv2.cvtColor(imgR_1, cv2.COLOR_BGR2GRAY)

stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL, imgR)
disparity = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
 
fig = plt.figure(constrained_layout=False, dpi=600)
plt.imshow(disparity, 'gray')
fig.savefig("./bm23.png",dpi=600)
#plt.jet()
plt.show()